# Synthetic image and cutout creation for Mosviz pipeline data

**Motivation**: The synthetic dataset we currently possess from the JWST data pipeline team for use in Mosviz contains simulated spectra from NIRSpec but no associated NIRCam photometry. We would like to have test imagery to display in Mosviz alongside these 2D and 1D spectra since we are not aware that the pipeline team has plans to produce any.

**Goal**: Populate an image of background noise with properly-scaled galaxy cutouts sourced from a Hubble Space Telescope image and placed at their analogous locations in the new image. These galaxies' real spectra do not necessarily correspond with those in our dataset, but we care more about the veneer of having photometry to match with our spectra in Mosviz at this point.

**Execution**: We pull our galaxy cutouts and catalog information from the Hubble Deep Field image. _(Formerly used ASTRODEEP's image of the [Abell 2774 Parallel](http://astrodeep.u-strasbg.fr/ff/?img=JH140?cm=grayscale) and [MACS J0416.1-2403 Parallel](http://astrodeep.u-strasbg.fr/ff/?ffid=FF_M0416PAR&id=1264&cm=grayscale).)_ We use `photutils` to select bright sources from the image for use in our own. We sought to use [Artifactory](https://bytesalad.stsci.edu/ui/repos/tree/General/jwst-pipeline%2Fdev%2Ftruth) to obtain a range of RA/Dec over which to project our synthetic image, but that information was absent. Instead, we place the image over a manually chosen RA/Dec range and place the cutouts in randomly selected locations within that field of view.

**Issues**:
- We wanted to scrape RA/Dec information from the data pipeline products to get a range of coordinates over which to scale our synthetic image, but it appears that the pipeline's data products lack `"TARG_RA"` or `"TARG_DEC"` keywords in their headers.
    - The data products also do not appear to have WCS information. Though it is not strictly needed to achieve this notebook's goals, it would be convenient to have.
- There does not appear to be an observation with Level 2 data in `jwst-pipeline/truth/test_nirspec_mos_spec2` and Level 3 data in `jwst-pipeline/truth/test_nirspec_mos_spec3`. All observations are either Level 2 only or Level 3 only.
- _(Resolved)_ A good number of the cutouts from the first couple of field images we tested had intrusions from other galaxies due to crowding. We settled on the Hubble Deep Field as a good source image, but had we not, we may have considered using galaxies modeled with Sersic profiles to get cleaner cutouts to inject into our synthetic image.

### Import packages

- `os` helps us create a new directory for the final cutouts from the synthetic image.
- `glob.glob` lists local files that match a given pattern.
- We use `astropy.units` to convert between degrees and arcseconds.
- `astropy.coordinates.SkyCoord` helps merge lists of right ascensions (RA) and declinations (Dec) into one object.
- `astropy.convolution.Gaussian2DKernel` smooths two-dimensional data by performing convolutions with two Gaussian functions.
- We use `astropy.io.fits` to read in existing FITS files and write a new one with the synthetic image.
- The objects from `astropy.nddata` assist in creating cutouts once we have identified galaxies to take from the field image and also with re-importing our synthetic image.
- The `sigma_*` methods from `astropy.stats` work with image data that is clipped to within a certain number of deviations from the mean. `gaussian_fwhm_to_sigma` is a float for converting full width at half maximum values to standard deviations from the image mean.
- The objects from `astropy.table` help with reading an modifying tabular data.
- We download a copy of the Artifactory spectra with `astropy.utils.data.download_file`.
- `astropy.wcs.WCS` creates a World Coordinate System object that is useful for transforming on-sky data from one patch to another.
- `pathlib.Path` allows for the creation of file paths that work across operating systems.
- The objects from `photutils.segmentation` do the work of analytically finding bright sources, separating those that overlap, and creating a catalog of the resulting information.
- We use `matplotlib.pyplot` to preview the field image, the cutouts, and various stages of our synthetic image.
- We use `numpy` to facilitate several specialized mathematical and array-related operations.
- `zipfile` assists with unzipping the downloaded Artifactory files.
- Finally, `%matplotlib inline` is notebook "magic" that helps display in-notebook plots consistently for different users.

In [ ]:
import os
import zipfile
from glob import glob
from pathlib import Path

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.convolution import Gaussian2DKernel
from astropy.io import fits
from astropy.nddata import block_reduce, Cutout2D, CCDData
from astropy.stats import (gaussian_fwhm_to_sigma,
                           sigma_clipped_stats, sigma_clip)
from astropy.table import Table, join
from astropy.utils.data import download_file
from astropy.wcs import WCS
from photutils.segmentation import (detect_sources, deblend_sources,
                                    SourceCatalog)

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
%matplotlib inline

### Download an HST galaxy field image

The galaxy cutouts come from the Hubble Deep Field image, whose header and flux data we save as separate variables.

In [ ]:
image_fits = fits.open('https://archive.stsci.edu/pub/hlsp/hdf/v2/mosaics/x4096/f814_mosaic_v2.fits')
image_header = image_fits[0].header
image_data = image_fits[0].data

image_data.shape

In [ ]:
imshow_params = {'cmap': 'bone', 'origin': 'lower'}
plt.imshow(image_data, vmin=0, vmax=image_data.mean()*3, **imshow_params)

We also save image statistics calculated from pixels within a chosen number of standard deviations from the image's mean intensity. Some of them are useful in creating the synthetic image later on.

In [ ]:
clipped_mean, clipped_median, clipped_stddev = sigma_clipped_stats(image_data,
                                                                   sigma=3.)

### Generate analytical galaxy cutouts

Whether or not the image comes with a catalog, we can use the methods imported from `photutils` to make our own `SourceCatalog`. We follow a workflow modified from `photuils`' [documentation on segmentation](https://photutils.readthedocs.io/en/stable/segmentation.html).

In [ ]:
npixels = 10 # minimum source length in pixels
sigma = 3 * gaussian_fwhm_to_sigma # from a FWHM of 3

# define a kernel for smoothing image data in the cutouts
kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
kernel.normalize()
#kernel = None

# take first pass at identifying sources
segments = detect_sources(data=image_data, threshold=8*clipped_stddev,
                          npixels=npixels**2, filter_kernel=kernel, mask=None)

# filter out any sources that are too close to the image border
segments.remove_border_labels(200)

In [ ]:
# separate overlapping sources into distinct entries
segm_deblend = deblend_sources(image_data, segments, npixels=npixels**2,
                               filter_kernel=kernel, nlevels=32,
                               contrast=1e-1)

# create an astropy Table of source information, sorted by area
sources = SourceCatalog(image_data, segm_deblend).to_table()
sources.sort('area', reverse=True)

We can also add a column to `sources` that contains RA/Dec information by creating a `WCS` object and using the field image's header in converting pixel locations to coordinates.

In [ ]:
image_wcs = WCS(image_header)

sources.add_columns(
    image_wcs.pixel_to_world_values(sources['xcentroid'], sources['ycentroid']),
    names=['RA', 'Dec'])

In [ ]:
sources[:5]

At this point, we are ready to ready to create and save cutouts using the source locations found earlier.

_Note: Depending on the value of `catalog_size`, the following cell can produce a lot of output. Right-click the cell and select "Disable Scrolling for Outputs" to expand it or "Enable Scrolling for Outputs" to condense it._

In [ ]:
# define parameters for cutout loop
all_cutouts = []
catalog_size = 33 # how many sources to include in the cutout list
cutout_length = 100 # the maximum length of a cutout in pixels
downsample_factor = 4 # scale sources down to proper size for the new image

for i in sources['label']:
    # save and plot the new cutout
    segm_source = segm_deblend.segments[i - 1]
    # (sources["label"] is 1-indexed, so subtract 1 for matching segm index)

    # implement the downsample
    cutout = segm_source.make_cutout(image_data, masked_array=True)
    cutout = block_reduce(cutout, downsample_factor) / downsample_factor**2
    
    plt.imshow(cutout, vmin=0, vmax=image_data.std()*3, **imshow_params)
    plt.show()
    
    all_cutouts.append(cutout)
    if len(all_cutouts) == catalog_size:
        break

___

### _[An alternate, catalog file-based cutout generation process]_

It is also possible to build a list of cutouts from an already-existing source catalog; this section serves as a reference for how it is done. Its cells are not necessary for the completion of the notebook and do not run due to the `%%script false --no-raise-error` statements at the top of each one.

Depending on the catalog, doing it this way can give you more detailed information about the resulting sources, but the process is less generalizable from one telescope's/instrument's/detector's images to another's. This path may also be less desirable for synthetic image creation because the cutouts will be square patches and thus look a little more, well, synthetic.

In [ ]:
%%script false --no-raise-error

# download sources' location and flux information
_source_info1 = Table.read('https://archive.stsci.edu/pub/hlsp/hdf/wfpc_hdfn_v2catalog/HDFN_wfpc_v2generic.cat',
                           format='ascii')
_source_info2 = Table.read('https://archive.stsci.edu/pub/hlsp/hdf/wfpc_hdfn_v2catalog/HDFN_f814_v2.cat',
                           format='ascii')
_sources = join(_source_info1, _source_info2)

# confirm that both tables contain the same objects in the same order (True)
( (_source_info1['NUMBER'] == _source_info2['NUMBER']).sum()
  == len(_source_info1)
  == len(_source_info2) )

In [ ]:
%%script false --no-raise-error

# sort sources by flux within 71.1 pixel diameter of source, or aperture 11
_sources.sort('FLUX_APER_11', reverse=True)

# filter out likely stars and sources with negative flux
_sources = _sources[(_sources['CLASS_STAR'] < .5)
                    & (_sources['FLUX_APER_8'] > 0)]

In [ ]:
%%script false --no-raise-error
_sources[:5]

In [ ]:
%%script false --no-raise-error

# convert the sources' WCS locations to in-image pixel values
_image_wcs = WCS(image_header)
_sources_x, _sources_y = image_wcs.world_to_pixel_values(_sources['ALPHA_J2000'],
                                                         _sources['DELTA_J2000'])

_Note: Depending on the value of `_catalog_size`, the following cell can produce a lot of output. Right-click the cell and select "Disable Scrolling for Outputs" to expand it or "Enable Scrolling for Outputs" to condense it._

In [ ]:
%%script false --no-raise-error

# define parameters for cutout loop
_cutout_list = []
_first_source = 0 # which source to cut out first (in descending order by flux)
_catalog_size = 20 # how many sources to include in the cutout list
_patch_length = 100 # the length of a cutout in pixels
_downsample_factor = 2

# save a list of good cutouts for later use
for x, y in list(zip(_sources_x, _sources_y))[_first_source:]:
    # use pixel locations to cut a source from the image
    _cutout = Cutout2D(image_data, (x, y),
                      _patch_length * _downsample_factor).data
    
    # bin by downsample_factor to increase field of view
    _cutout = block_reduce(_cutout, _downsample_factor)
    
    # skip any cutouts that extend past the image border
    if (  np.all(_cutout[-1] <= 0) or np.all(_cutout[0] <= 0)
          or np.all(_cutout[:,-1] <= 0) or np.all(_cutout[:,0] <= 0)  ):
        continue
        
    # save and plot the new cutout
    _cutout_list.append(_cutout)
    
    plt.imshow(_cutout, vmin=-1e-5, vmax=image_data.std(), **imshow_params)
    plt.show()
    
    if len(_cutout_list) == _catalog_size:
        break

___

### (Try to) Extract destination RA/Dec from spectra files

We download copies of Level 3 data products from the JWST pipeline dated May 19, 2021 and hosted in a STScI Box folder. (For more current files, visit the [Artifactory](https://bytesalad.stsci.edu/ui/repos/tree/General/jwst-pipeline%2Fdev%2Ftruth%2Ftest_nirspec_mos_spec3.).) The files are saved in this notebook's current location. If you prefer to save them elsewhere, change `filepath`.

In [ ]:
filepath = Path('./')

# If necessary, download and extract spectra files
spec_link = 'https://stsci.box.com/shared/static/3rz8g1fizu6dh7zdymxouvdd8usnm3jc.zip'
spec_zip = filepath / 'artifactory_spectra-2021-05-19.zip'
spec_path = filepath / spec_zip.stem

if spec_path.exists():
    print('Spectra already on disk.')
else:
    if not (filepath / spec_zip).exists():
        print(f"Downloading '{spec_zip.name}'.")
        box_file = download_file(spec_link, cache=True)
        
        # place link to downloaded file in current directory
        os.symlink(box_file, filepath / spec_zip)
    
    print('Extracting spectra.')
    
    # create target directory and extract spectra files 
    (filepath / spec_path).mkdir()
    with zipfile.ZipFile(filepath / spec_zip, 'r') as zf:
        zf.extractall(spec_path)

In [ ]:
# view level 3 spectra FITS header information
x1d_header = fits.getheader(spec_path / 'jw00626-o030_s00000_nirspec_f170lp-g235m_x1d.fits', ext=1)
s2d_header = fits.getheader(spec_path / 'jw00626-o030_s00000_nirspec_f170lp-g235m_s2d.fits', ext=1)

In [ ]:
(x1d_header['SRCRA'], x1d_header['SRCDEC']), (s2d_header['SRCRA'], s2d_header['SRCDEC'])

In [ ]:
WCS(x1d_header)

This header's listed source RA/Dec of (0, 0) seems suspicious. Examining all of this observation's file headers reveals that it is indeed not in line with the other pointings.

In [ ]:
# search for RA/Dec information from Artifactory observation files
x1d_header_list = [fits.getheader(file, ext=1)
                   for file in spec_path.glob('jw00626*x1d.fits')]

_ras, _decs = np.array([[h['SRCRA'], h['SRCDEC']] for h in x1d_header_list]).T
_ras, _decs

### Create synthetic image

To get a more realistic field of view, we randomly generate our sources' RA/Dec information in a predetermined patch of sky. We choose the patch size by approximating NIRSpec's 3.6'x3.4' Micro-Shutter Assembly (MSA) to a square field of view.

In [ ]:
# (ranges based on NIRSpec MSA's on-sky projection size of 3.6x3.4 arcmins)
np.random.seed(19)
ras = np.random.uniform(0, 1/15, catalog_size)
decs = np.random.uniform(-1/30, 1/30, catalog_size)

We initialize a `numpy` array and fill it with a normally-distributed background noise level based on some of the clipped image statistics that were calculated earlier.

In [ ]:
# create synthetic image onto which cutouts will be pasted
synth_img_size = 1000
synth_image = np.zeros((synth_img_size, synth_img_size))

In [ ]:
# add noise
synth_image += np.random.normal(loc=clipped_mean, scale=clipped_stddev*8,
                                size=synth_image.shape)

In [ ]:
plt.imshow(synth_image, **imshow_params)
plt.show()

### Fill out new WCS object for `synth_image`

Creating a `WCS` object for `synth_image` allows it to be mathematically transformed into a projection on the sky. That projection can then be compared to other FITS images with their own WCS information.

In [ ]:
synth_wcs = WCS(naxis=2)

The next step is to calculate field of view information for `synth_image`.

In [ ]:
# find the range of sources in RA and dec
ra_bounds = np.array([ras.max(), ras.min()])
dec_bounds = np.array([decs.max(), decs.min()])

delta_ra = np.ptp(ras)
delta_dec = np.ptp(decs)

In [ ]:
# save the maximum span in coordinates, RA or dec
if delta_ra > delta_dec:
    min_image_fov = abs(delta_ra * np.cos(np.pi / 180 * dec_bounds.sum() / 2))
else:
    min_image_fov = delta_dec
    
min_image_fov

In [ ]:
# scale this field of view (FOV) by pixels
pix_scale = min_image_fov / synth_img_size

# add a buffer to the FOV's borders to prevent clipping sources
pix_scale *= 1.2
(pix_scale * u.deg).to(u.arcsecond)

With those calculations done, the `WCS` object is ready to be filled out.

In [ ]:
synth_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']

# match value of center pixel of detector to value of FOV's central coordinate in the sky
synth_wcs.wcs.crpix = [synth_img_size / 2, synth_img_size / 2]
synth_wcs.wcs.crval = [ra_bounds.sum() / 2, dec_bounds.sum() / 2]

# distance (in sky coordinates) traversed by one pixel length in each dimension
synth_wcs.wcs.cdelt = [-pix_scale, pix_scale]

synth_wcs

In [ ]:
# convert source RAs/decs from real coordinates to pixels 
ras_pix, decs_pix = np.round(synth_wcs.world_to_pixel_values(ras, decs)).astype(int)
ras_pix, decs_pix

### Populate `synth_image` with the cutouts

Now is the time to add the cutouts into `synth_image` and complete its creation.

In [ ]:
for i, co in enumerate(all_cutouts):
    # first, calculate cutout's distances from center to each edge, accounting
    # for odd numbers by using np.floor to shift that axis' values by 0.5
    cutout_ranges = np.array([np.floor([-n/2, n/2]) for n in co.shape],
                             dtype='int')
    
    # remember to flip RA/dec to dec/RA when indexing image array
    synth_image[
        decs_pix[i] + cutout_ranges[0,0] : decs_pix[i] + cutout_ranges[0,1],
        ras_pix[i] + cutout_ranges[1,0] : ras_pix[i] + cutout_ranges[1,1]
    ] += co

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(synth_image, vmin=0, vmax=synth_image.std(), **imshow_params)
#ax.imshow(synth_image, vmin=0, vmax=synth_image.mean()*3, **imshow_params)
ax.tick_params(labelsize=14)
plt.show()

We save the image to a local location. The default is this notebook's current directory; you can change that by adjusting `savepath` to your preferred path.

This cell will overwrite a same-named file if it already exists in that location.

In [ ]:
savepath = Path('./')
synth_file = 'synthetic_mosviz_image.fits'

fits.writeto(savepath / synth_file, synth_image,
             header=synth_wcs.to_header(), overwrite=True)

### Harvest cutouts from synthetic image

Finally, we can create image cutouts from the new synthetic image to associate with the pipeline's synthetic spectral data in [Mosviz](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html). We begin by reopening the synthetic image with `astropy`'s `CCDData` class in order to handle the WCS information in its header more smoothly.

Then, we use `Cutout2D` to create square cutouts of each source in the image by using coordinate information previously taken from the synthetic sprectra files.

In [ ]:
ccd_synth_img = CCDData.read(savepath / synth_file, unit='electron/s')

In [ ]:
synth_cutout_size = 50 # pixels
synth_cutout_list = [Cutout2D(ccd_synth_img, SkyCoord(r, d), synth_cutout_size)
                     for r, d in zip(ras * u.deg, decs * u.deg)]

plt.imshow(synth_cutout_list[2].data, **imshow_params,
           vmin=0, vmax=ccd_synth_img.data.std())

These cutout images are also ready for saving after adding some extra information to their headers. It may be useful to create a new folder and save them there by modifying the `savepath2` variable below.

_Note: Depending on the length of `synth_cutout_list`, the following cell can produce a lot of output. Right-click the cell and select "Disable Scrolling for Outputs" to expand it or "Enable Scrolling for Outputs" to condense it._

In [ ]:
cutout_dir = savepath / 'mosviz-cutouts/'
if not (savepath / cutout_dir).exists():
    cutout_dir.mkdir()

for i, cut in enumerate(synth_cutout_list):
    sdt = cut.data
    shd = cut.wcs.to_header()
    
    # add metadata for proper reading in Mosviz
    shd['OBJ_RA'] = (ras[i], 'Source right ascension')
    shd['OBJ_DEC'] = (decs[i], 'Source declination')
    shd['OBJ_ROT'] = (0, '')
    
    fits.writeto(cutout_dir / f"synth_cutout{i}.fits",
                 sdt, header=shd, overwrite=True)
    
    plt.imshow(sdt, **imshow_params, vmin=0, vmax=ccd_synth_img.data.std())
    plt.show()

These cutout files and the spectral files we examined earlier can serve as the data behind a workflow in [Mosviz](https://jdaviz.readthedocs.io/en/latest/mosviz/index.html). Please see other notebooks in this directory for more information on how to use that tool.

<p>
    <span style="line-height: 60px;"> <i> Authors: O. Justin Otor and Robel Geda </i> </span>
    <img style="float: right;/* clear: right; */vertical-align: text-bottom;display: inline-block;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px">
</p>

-----